In [1]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5f37c74a4bd1a2394b4fa5dcbb449244bb183873f839c409c1c2b5d3191175e6de6040979aeb78f7894665a168392bc55c2d58d905f027b397'

In [2]:
import aocd
from aocd.models import Puzzle
day = 17
year = 2024
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [3]:
test_data = """0,1,2,3"""
data_test = test_data.splitlines()

In [4]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(5,
 ['Register A: 37283687',
  'Register B: 0',
  'Register C: 0',
  '',
  'Program: 2,4,1,3,7,5,4,1,1,3,0,3,5,5,3,0'])

In [298]:
class CPU():
    opmap = {
        0: "adv",
        1: "bxl",
        2: "bst",
        3: "jnz",
        4: "bxc",
        5: "out",
        6: "bdv",
        7: "cdv",
    }
    
    def __init__(self, program: str="", A=0, B=0, C=0, debug=False, replicate=False):
        self.A = A
        self.B = B
        self.C = C
        self.ip = 0
        self.instructions = []
        self.Ni = 0
        self.debug = debug
        self.output = []
        self.exit = False
        self.replicate = replicate
        if program:
            self.load(program)
            
    def load(self, program):
        self.instructions = list(map(int, program.split(',')))
        self.Ni = len(self.instructions)
        
    def get_combo_operand(self, operand):
        # if operand == 7:
        #     raise ValueError
        match operand:
            case 4:
                operand = self.A
            case 5:
                operand = self.B
            case 6:
                operand = self.C
        return operand
    
    def execute(self, op, operand):
        combo_operand = self.get_combo_operand(operand)
        match op:
            case 0:
                self._adv(combo_operand)
            case 1:
                self._bxl(operand)
            case 2:
                self._bst(combo_operand)
            case 3:
                self._jnz(operand)
            case 4:
                self._bxc(operand)
            case 5:
                self._out(combo_operand)
            case 6:
                self._bdv(combo_operand)
            case 7:
                self._cdv(combo_operand)
    
    def state(self, end="\n"):
        if self.debug:
            print(f"A: {self.A}, \tB: {self.B}, \t\tC: {self.C}", end=end)
                
    def run(self):
        while self.ip <= self.Ni-1:
            op, operand = self.instructions[self.ip: self.ip+2]
            if self.debug:
                print(f"executing: {CPU.opmap[op]} <- {operand} \t", end="")
            self.state()
            self.execute(op, operand)
            # if self.exit:
            #     return False
            self.ip += 2
        return True
    
    def reset(self, A=0, B=0, C=0):
        self.A = A
        self.B = B
        self.C = C
        self.exit = False
        self.ip = 0
        self.output = []
    
    def _adv(self, o):
        self.A = int(self.A / pow(2, o))
    
    def _bxl(self, o):
        self.B = self.B ^ o
    
    def _bst(self, o):
        self.B = o % 8
    
    def _jnz(self, o):
        if self.A != 0:
            self.ip = o-2
            
    def _bxc(self, o):
        self.B = self.B ^ self.C
        
    def _out(self, o):
        out = o%8
        # print("out: ", out, end='\n')
        self.output.append(out)
        if self.replicate:
            if self.output[-1] != self.instructions[len(self.output)-1]:
                self.exit = True
    
    def _bdv(self, o):
        self.B = int(self.A / pow(2, o))
        
    def _cdv(self, o):
        self.C = int(self.A / pow(2, o))
        

In [211]:
A, data = 37283687, """2,4,1,3,7,5,4,1,1,3,0,3,5,5,3,0"""
# A, data = 729, """0,1,5,4,3,0"""

In [169]:
cpu = CPU(program=data, A=A, debug=False)

In [170]:
cpu.run()

True

In [171]:
res = ",".join(map(str, cpu.output))
res

'1,5,3,0,2,5,2,5,3'

In [103]:
puzzle.answer_a = res

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


In [214]:
# Part 2

In [546]:
# extract algorithm
A = 18323965
B = ((A%8) ^ 3)
C = A // 2**B
B = B ^ C
B = B ^ 3
A = A // 2**3
out = B % 8
# conclusion: after each loop we strip the last 3-bits and to get the full output we need the loop to run 15 times: 46 to 48 bit number
# we can construct a possible solution by iterating over all possible 3-bit numbers and construct the output in reverse

A, B, C, out


(2290495, 286306, 286311, 2)

In [250]:
# debug computation
def comp(A):
    B = ((A%8) ^ 3)
    # C = A // 2**B
    B = B ^ (A // 2**B)
    B = B ^ 3
    A = A // 2**3
    return B % 8


In [548]:
%%time
i = 0
longest_match = 0
cpu = CPU(program=data, A = i, replicate=False)
# cpu = CPU(program="0,3,5,4,3,0", A = i, replicate=True)

N = len(cpu.instructions)
stack = [[]]
results = []
while len(stack):
    res = stack.pop()
    i = len(res) + 1
    for j in range(8):
        binj = format(j, '03b')
        num = "".join(res) + binj
        num = num.ljust(48, '1')
        # print(num)
        cpu.reset(A=int(num, 2))
        cpu.run()
        if (len(cpu.output) == len(cpu.instructions)) and (cpu.output[-i] == cpu.instructions[-i]):
            # print(j, i, binj, num, "!")
            # print(cpu.output, cpu.instructions, res)
            new_res = res + [binj]
            if i == 16 and (cpu.output[0] == cpu.instructions[0]):
                results.append(new_res)
            else:
                # print(res)
                stack.append(new_res)


CPU times: user 110 ms, sys: 3 µs, total: 110 ms
Wall time: 109 ms


In [549]:
candidates = ["_".join(x) for x in results]
sorted(candidates)

['011_000_100_101_001_011_000_001_000_101_111_001_100_111_111_101']

In [544]:
# cpu = CPU(program=data, A = 18323965, replicate=True, debug=True)
cpu = CPU(program=data, A = int(sorted(candidates)[0], 2), replicate=True, debug=False)
cpu.run()
print(cpu.output)
print(cpu.instructions)

[2, 4, 1, 3, 7, 5, 4, 1, 1, 3, 0, 3, 5, 5, 3, 0]
[2, 4, 1, 3, 7, 5, 4, 1, 1, 3, 0, 3, 5, 5, 3, 0]


In [539]:
puzzle.answer_b = int(sorted(candidates)[0], 2)

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 17! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [550]:
int(sorted(candidates)[0], 2)

108107566389757

In [551]:
candidates

['011_000_100_101_001_011_000_001_000_101_111_001_100_111_111_101']

In [556]:
import re

a, b, c, *prog = map(int, re.findall(r'\d+', 
                     open('./data/input_17').read()))

def eval(a, b, c, i=0, R=[]):
    while i in range(len(prog)):
        C = {0:0,1:1,2:2,3:3,4:a,5:b,6:c}

        match prog[i:i+2]:
            case 0, op: a = a >> C[op]
            case 1, op: b = b ^ op
            case 2, op: b = 7 & C[op]
            case 3, op: i = op-2 if a else i
            case 4, op: b = b ^ c
            case 5, op: R = R + [C[op] & 7]
            case 6, op: b = a >> C[op]
            case 7, op: c = a >> C[op]
        i += 2
    return R

print(*eval(a,b,c), sep=',')


def find(a, i):
    if eval(a, b, c) == prog: print(a)
    if eval(a, b, c) == prog[-i:] or not i:
        for n in range(8): find(8*a+n, i+1)

find(0, 0)

1,5,3,0,2,5,2,5,3
108107566389757


In [553]:
prog

[2, 4, 1, 3, 7, 5, 4, 1, 1, 3, 0, 3, 5, 5, 3, 0]

In [561]:
find(0, 0)

108107566389757


In [562]:
# cpu = CPU(program=data, A = 18323965, replicate=True, debug=True)
cpu = CPU(program=data, A = 0b011_000_100_101_001_011_000_001, replicate=True, debug=False)
cpu.run()
print(cpu.output)
print(cpu.instructions)

[1, 3, 0, 3, 5, 5, 3, 0]
[2, 4, 1, 3, 7, 5, 4, 1, 1, 3, 0, 3, 5, 5, 3, 0]
